##Tuning hyperparameter (hyperopt)

In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure

import os 
import datetime

%load_ext tensorboard

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials,fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix_three/DS_road_signs'

/content/drive/My Drive/Colab Notebooks/matrix_three/DS_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics = ['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size',128), 
      epochs = params_fit.get('epochs',5),
      verbose = params_fit.get('verbose',1),
      validation_data = params_fit.get('validation_data',(X_train,y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, X_test,y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation ='relu',input_shape = input_shape), 
    Conv2D(filters = 32, kernel_size=(3,3), activation ='relu', padding = 'same'), 
    Conv2D(filters = 32, kernel_size=(3,3), activation ='relu', padding = 'same'),               
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters = 64, kernel_size=(3,3), activation ='relu', padding = 'same'), 
    Conv2D(filters = 64, kernel_size=(3,3), activation ='relu', padding = 'same'), 
    Conv2D(filters = 64, kernel_size=(3,3), activation ='relu'),             
    MaxPool2D(),
    Dropout(0.3), 

    Conv2D(filters = 128, kernel_size=(3,3), activation ='relu', padding = 'same'),
    Conv2D(filters = 128, kernel_size=(3,3), activation ='relu', padding = 'same'),  
    Conv2D(filters = 128, kernel_size=(3,3), activation ='relu'),             
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes,activation='softmax'),
    
])

#get_cnn_v4(input_shape, num_classes).summary()
#train_and_predict(get_cnn_v5(input_shape,num_classes))

In [30]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 8s 31ms/step - loss: 2.1527 - accuracy: 0.3933 - val_loss: 0.5562 - val_accuracy: 0.8118
Epoch 2/5
272/272 [==============================] - 5s 19ms/step - loss: 0.4416 - accuracy: 0.8629 - val_loss: 0.1049 - val_accuracy: 0.9701
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1962 - accuracy: 0.9412 - val_loss: 0.0446 - val_accuracy: 0.9872
Epoch 4/5
272/272 [==============================] - 5s 19ms/step - loss: 0.1353 - accuracy: 0.9614 - val_loss: 0.0270 - val_accuracy: 0.9921
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1093 - accuracy: 0.9686 - val_loss: 0.0274 - val_accuracy: 0.9918


0.9707482993197278

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation ='relu',input_shape = input_shape), 
    Conv2D(filters = 32, kernel_size=(3,3), activation ='relu', padding = 'same'),                
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters = 64, kernel_size=(3,3), activation ='relu', padding = 'same'),  
    Conv2D(filters = 64, kernel_size=(3,3), activation ='relu'),             
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters = 128, kernel_size=(3,3), activation ='relu', padding = 'same'),  
    Conv2D(filters = 128, kernel_size=(3,3), activation ='relu'),             
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes,activation='softmax'),
    
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics = ['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size',128)), 
      epochs = 5,
      verbose = 0

  )

  score = accuracy = model.evaluate(X_test, y_test,verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}


In [28]:
space = {
    'batch_size' : hp.quniform('batch_size',100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4380845995149134, 'dropout_cnn_block_three': 0.489935947957088, 'dropout_cnn_block_two': 0.43681372132590557, 'dropout_dense_block_one': 0.34353449796060476, 'dropout_dense_block_two': 0.3774142304051271}
accuracy=0.9077097773551941
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4988503687921654, 'dropout_cnn_block_three': 0.44723501336706184, 'dropout_cnn_block_two': 0.34685381941377647, 'dropout_dense_block_one': 0.625487546281321, 'dropout_dense_block_two': 0.6341334753517158}
accuracy=0.8743764162063599
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.42897912942238303, 'dropout_cnn_block_three': 0.4082582736853114, 'dropout_cnn_block_two': 0.4318868135991615, 'dropout_dense_block_one': 0.4859100654346538, 'dropout_dense_block_two': 0.6228254287971783}
accuracy=0.9244897961616516
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4243266376545871, 'dropout_cnn_block_three': 0.3262637897795007, 'dropout_cnn_block_two': 0.3069462512818